In [ ]:
# !pip install pandas seaborn matplotlib
# !pip install optuna
# !pip install -U tqdm
# !pip install scipy
# !pip install pwlf

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import numpy as np
import pandas as pd
import pwlf
import seaborn as sns
import tqdm

from math import tau

sns.set_style('whitegrid')

# dataset introspection

In [ ]:
D= pd.read_csv('~/.ros/tuning_curve_samples.csv')
# D= pd.read_csv('../../../tuning_curve_samples mfj4 rfj4 lfj4 th4 th5.csv')

print(f"found {len(D) / len(D['joint'].unique())} samples")
# drop J1s but keep WRJ1. The others don't exist
D= D[
    (D['joint'] != 'FFJ1') &
    (D['joint'] != 'MFJ1') &
    (D['joint'] != 'RFJ1') &
    (D['joint'] != 'LFJ1') &
    (D['joint'] != 'THJ1')
    ]
D.describe()

In [ ]:
# facet grid by joint
plt.figure(figsize=(15, 10), dpi=150)

# like this, but wrap in columns instead of rows
g = sns.FacetGrid(D, col='joint', col_wrap=5, height=4, sharex=False, sharey=False, hue='joint')

g.map(sns.scatterplot, 'reading', 'position', linewidth = 0)
for ax in g.axes.flat:
    ax.xaxis.set_major_locator(plticker.MultipleLocator(100))
    # format, e.g.,  1200 as 1.2k
    ax.xaxis.set_major_formatter(plticker.FuncFormatter(lambda x, _: f'{x/1000:.1f}k'))
    # rotate labels
    for label in ax.get_xticklabels():
        label.set_rotation(60)
    ax.yaxis.set_major_locator(plticker.MaxNLocator(20))
    ax.yaxis.set_major_formatter(plticker.FormatStrFormatter('%.2f'))

g.savefig(f'readings-vs-positions {pd.Timestamp.now()}.svg')

In [ ]:
# facet stripplot by joint
g = sns.FacetGrid(D, col='joint', col_wrap=5, height=4, sharex=False, sharey=False, hue='joint')
g.map(sns.stripplot, 'reading', order= None)
# g.savefig(f'stripplot-readings {pd.Timestamp.now()}.svg')

# fit piecewise linear function

In [ ]:
import pwlf
def fit(x,y,N):
    model = pwlf.PiecewiseLinFit(x, y)
    model.fit(N)
    xb = model.fit_breaks
    yb = model.predict(xb)
    return xb, yb

In [ ]:
FINGER = 'LFJ5'
Df = D[D['joint'] == FINGER]
Df = Df.sort_values(by='reading')

# filter broken datapoints
Df = Df[Df['reading'] != 0]
# Df = Df[Df['position'] < 0.2]
# drop points with reading between 1750 and 1875 with position > -0.275
# Df = Df[~((Df['reading'] > 2600) & (Df['reading'] < 2700) & (Df['position'] > 0.35))]
# Df = Df[~((Df['reading'] > 2670) & (Df['reading'] < 2700) & (Df['position'] > 0.3))]
# Df = Df[~((Df['reading'] > 2600) & (Df['reading'] < 2700) & (Df['position'] < 0.2))]
# Df = Df[~(Df['position'] < -0.28)]
# Df = Df[~((Df['reading'] < 1500) & (Df['position'] < 0.1))]

# Df = Df.drop(Df.index[[9, 11]])

x = Df['reading']
y = Df['position']

Df.plot.scatter(x='reading', y='position', title=FINGER)
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 10), dpi=100)
fig.suptitle(f'{FINGER} readings vs positions')

for i, ax in tqdm.tqdm(list(enumerate(axs.flat))):
    N = i + 1
    xb, yb = fit(x, y, N)
    ax.plot(xb, yb, 'r-o')
    ax.scatter(x, y)
    ax.set_title(f'N={N}')
    ax.set_xlabel('reading')
    ax.set_ylabel('position')
    ax.xaxis.set_major_locator(plticker.MultipleLocator(100))
    ax.yaxis.set_major_locator(plticker.MaxNLocator(20))
    ax.yaxis.set_major_formatter(plticker.FormatStrFormatter('%.2f'))

plt.savefig(f'piecewise-linear-fits {pd.Timestamp.now()}.svg')

In [ ]:
# hand-picked number of segments:
N = 2

xb,yb = fit(x,y,N)
plt.figure(figsize=(15, 10), dpi=50)
plt.scatter(x, y)
plt.plot(xb, yb, label=f"{N} segment linear fit", color='black')
plt.title(FINGER)
plt.xlabel('reading')
plt.ylabel('position')
plt.legend()
plt.savefig(f'{FINGER} {N} segments {pd.Timestamp.now()}.svg')

# calibration file format for shadow hand tuning curves
entries = f'["{FINGER}", [' + ", ".join([f"[{x:.2f}, {y*360/tau:.2f}]" for x,y in zip(xb,yb)]) + "]],"
print(f"Copy this line to the corresponding calibration.yaml in sr_hand_config:\n" + entries)